# CSE 168 Lab 3 - Option 1

This notebook describes CSE 168 Lab 3 - Option 1 for students Shawn Duong, Chloe Engel, Charison Gill-Branion, and Isabella Montoya in the Fall semester of 2022.

For this lab, we are training a model to detect 7 hand gestures. This follows the tutorial given in the lab handout by Nicholas Renotte.

Before running this notebook, one should set up the venv and install the dependencies as per the tutorial:

```
python -m venv tfod

source tfod/bin/activate # Linux
.\tfod\Scripts\activate # Windows 

python -m pip install --upgrade pip
pip install ipykernel
python -m ipykernel install --user --name=tfodj
```

Make sure that the notebook's kernel is tfodj as well.

# Part 1: Collecting Training Images

## Step 1: Install and Import Dependencies

We must install and import the dependencies. We need opencv-python in order to use computer vision related functionalities.

In [1]:
!pip install opencv-python

In [1]:
import cv2
import os
import time

## Step 2: Define the Images to Collect

We are collecting the hand gestures that make up "Hello World," and saving 5 images per gesture. We can collect more training images by just re-running the code in step 4, though.

In [3]:
# The gestures we are training the model to detect.
labels = ["h", "e", "l", "o", "w", "r", "d"]

# The number of training images per gesture we will take.
nImgs = 5

## Step 3: Set Up File Structure

We are going to save everything in `./tensorflow/workspaces/images/training_images/`.

In [4]:
# The path to store our training images in.
path = "./tensorflow/workspace/images/training_images/"

# Create the path if it does not exist.
if not os.path.exists(path):
    os.makedirs(path)

## Step 4: Capture Training Images From Webcam

We capture 5 images per gesture from the webcam. We can press 'q' on our keyboard to quit early, or 'c' to capture an image when we're ready.

In [5]:
# If we have already collected images, we may want to skip this step.
# If you have not collected images yet, set this to False.
skipTraining = True

if not skipTraining:
    
    cap = cv2.VideoCapture(0)

    # Loop for all gestures we want to train.
    for label in labels:

        print(f"Capturing images for: {label}")

        completed = 0
        earlyExit = False

        # Loop for however many images we wish to capture per gesture.
        while completed < nImgs:

            # Read from the camera and show it to us.
            _, frame = cap.read()

            try:
                cv2.imshow("Frame", frame)
            except:
                continue

            # Webcam refresh rate.
            time.sleep(0.01)

            # Press 'q' to quit.
            key = cv2.waitKey(1) & 0xFF
            if key == ord('q'):
                earlyExit = True
                break

            # Press 'c' to capture.
            elif key == ord('c'):
                # Capture and save the image.
                completed += 1
                print(f"Capturing image {completed}/{nImgs}")
                cv2.imwrite(path+f"{label}_{int(time.time())}.jpg", frame)

        if earlyExit:
            break

    cap.release()
    cv2.destroyAllWindows()

## Step 5: Segment the Images Into Training and Testing

Annotate the images with labelimg, and then move some into training and others into testing along with their annotations.

# Part 2: Training and Detection

## Step 1: Download and Compile TFOD

We must download and install TFOD from TensorFlow's GitHub. We must compile all the proto files to do so. This is different for Linux and Windows.

In [ ]:
# Make the repository where we will clone the TensorFlow models repo.
if not os.path.exists("./models/"):
    os.makedirs("./models/")
    !git clone https://github.com/tensorflow/models ./models/
    
# For Linux.
if os.name == "posix":
    
    # For Arch Linux.
    if "arch" in os.uname().release:
        !pacman -Syu protobuf
        
    # If you're not using Arch, you're probably on Ubuntu or some
    # other Debian derivative and use apt.
    else:
        !apt-get install protobuf-compiler
        
    # Compile the proto files.
    !cd ./models/research && protoc object_detection/protos/*.proto --python_out=. \
     && cp object_detection/packages/tf2/setup.py . && python -m pip install .

# For Windows.
else:
    # TODO.
    protoc_path = "tensorflow\\protoc\\"
    os.makedirs(protoc_path, exist_ok=True)
    
    !pip install wget
    import wget
    
    # TODO: Clean
    !cd {protoc_path} && dir && del /f "protoc-3.15.6-win64.zip"
    
    protoc_url="https://github.com/protocolbuffers/protobuf/releases/download/v3.15.6/protoc-3.15.6-win64.zip"
    wget.download(protoc_url, out=protoc_path)
    
    !cd {protoc_path} && tar -xf protoc-3.15.6-win64.zip
    
    # PATH manual reset
    #os.environ['PATH'] = "\\Users\\chari\\anaconda3\\Lib\\site-packages\\cv2\\../../x64/vc14/bin;C:\\WINDOWS\\system32;C:\\WINDOWS;C:\\WINDOWS\\System32\\Wbem;C:\\WINDOWS\\System32\\WindowsPowerShell\\v1.0\\;C:\\WINDOWS\\System32\\OpenSSH\\;C:\\Program Files\\Git\\cmd;C:\\Program Files\\MATLAB\\R2021a\\bin;C:\\Program Files (x86)\\dotnet\;C:\\Program Files\\Docker\\Docker\\resources\\bin;C:\\ProgramData\\DockerDesktop\\version-bin;C:\\Users\\chari\\anaconda3;C:\\Users\\chari\\anaconda3\\Library\\mingw-w64\\bin;C:\\Users\\chari\\anaconda3\\Library\\usr\\bin;C:\\Users\\chari\\anaconda3\\Library\\bin;C:\\Users\\chari\\anaconda3\\Scripts;C:\\python\\3.10.1\\Scripts\\;C:\\python\\3.10.1\\;C:\\Users\\chari\\AppData\\Local\\Microsoft\\WindowsApps;C:\\Users\\chari\\AppData\\Local\\Programs\\Microsoft VS Code\\bin;"
    
    # Add protoc to path
    #os.environ['PATH'] += os.path.abspath(protoc_path + "bin;")
    #print(os.environ['PATH'])
    
    !cd ./models/research && protoc object_detection/protos/*.proto --python_out=. && copy object_detection\\packages\\tf2\\setup.py setup.py
    !cd ./models/research/slim && pip install -e .

error: you cannot perform this operation unless you are root.
Processing /home/skat/doc/repos/cse168-final/models/research
  Preparing metadata (setup.py) ... done


INFO: pip is looking at multiple versions of tabulate to determine which version is compatible with other requirements. This could take a while.


INFO: pip is looking at multiple versions of requests to determine which version is compatible with other requirements. This could take a while.
  Using cached requests-2.28.1-py3-none-any.whl (62 kB)
INFO: pip is looking at multiple versions of regex to determine which version is compatible with other requirements. This could take a while.
  Using cached regex-2022.10.31-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (770 kB)
INFO: pip is looking at multiple versions of pyyaml to determine which version is compatible with other requirements. This could take a while.
  Using cached PyYAML-5.4.1-cp310-cp310-linux_x86_64.whl
INFO: pip is looking at multiple versions of pytz to determine which version is compatible with other requirements. This could take a while.
  Using cached pytz-2022.6-py2.py3-none-any.whl (498 kB)
INFO: pip is looking at multiple versions of python-dateutil to determine which version is compatible with other requirements. This could take a while.
  Using

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.1/43.1 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.1/43.1 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.0/43.0 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.0/43.0 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.0/43.0 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Step 2: Install TensorFlow and Upgrade Protobuf

We will be using TensorFlow, so we should make sure it is installed before proceeding. We should also upgrade protobuf, since older versions may lead to an error about `builder.py`.

In [ ]:
# You probably already have this installed, but just in case.
!pip install tensorflow
!pip install protobuf==3.20

## Step 3: Get The Pretrained Model

A pretrained model is available at http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz. We just need to get it and extract it now before we can use it with our object detection.

In [ ]:
url = "http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz"

# Make the directory.
if not os.path.exists("./tensorflow/workspace/pretrained_models/"):
    os.makedirs("./tensorflow/workspace/pretrained_models/")

# For Linux.
if os.name == "posix":
    !wget {url}
    !mv ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz ./tensorflow/workspace/pretrained_models/
    !cd ./tensorflow/workspace/pretrained_models/ && tar xzvf ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz
    
# For Windows.
else:
    # TODO.
    pass

## Step 4: Create the Label Map

We create a file `./tensorflow/workspace/annotations/label_map.pbtxt` containing our labels.

In [ ]:
labels = [
    {"name": "h", "id": 1},
    {"name": "e", "id": 2},
    {"name": "l", "id": 3},
    {"name": "o", "id": 4},
    {"name": "w", "id": 5},
    {"name": "r", "id": 6},
    {"name": "d", "id": 7},
]

data = ""

for label in labels:
    
    name = label["name"]
    idno = label["id"]
    
    data += "item {\n"
    data += f"\tname: '{name}'\n"
    data += f"\tid: {idno}\n"
    data += "}\n"

if not os.path.exists("./tensorflow/workspace/annotations/"):
    os.makedirs("./tensorflow/workspace/annotations/")
    
with open("./tensorflow/workspace/annotations/label_map.pbtxt", "w") as f:
    f.write(data)

## Step 5: Copy the Model Config to the Training Folder

We need to create a training folder and copy the model config over to it before we begin training.

In [ ]:
if not os.path.exists("./tensorflow/workspace/models/model/"):
    os.makedirs("./tensorflow/workspace/models/model/")

# Linux.
if os.name == "posix":
    !cp ./tensorflow/workspace/pretrained_models/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config \
     ./tensorflow/workspace/models/model/
    
# Windows.
else:
    # TODO.
    pass

## Step 6: Create TF Records

We clone and run the author's scripts to generate the TF records used in the next step.

In [ ]:
if not os.path.exists("GenerateTFRecord"):
    !git clone https://github.com/nicknochnack/GenerateTFRecord

!python GenerateTFRecord/generate_tfrecord.py -x "./tensorflow/workspace/images/train" \
 -l "./tensorflow/workspace/annotations/label_map.pbtxt" -o "./tensorflow/workspace/annotations/train.record"
!python GenerateTFRecord/generate_tfrecord.py -x "./tensorflow/workspace/images/test" \
 -l "./tensorflow/workspace/annotations/label_map.pbtxt" -o "./tensorflow/workspace/annotations/test.record"

## Step 7: Import Everything and Update the Config for Transfer Learning

We should import everything needed for training and detection now. We can update the config for transfer learning with our training images. If you get warnings about CPU optimization, ignore them -- it has to do with your hardware.

In [ ]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [ ]:
config = config_util.get_configs_from_pipeline_file("./tensorflow/workspace/models/model/pipeline.config")
pconfig = pipeline_pb2.TrainEvalPipelineConfig()

with tf.io.gfile.GFile("./tensorflow/workspace/models/model/pipeline.config", "r") as f:
    pstr = f.read()
    text_format.Merge(pstr, pconfig)
    
pconfig.model.ssd.num_classes = len(labels)
pconfig.train_config.batch_size = 4
pconfig.train_config.fine_tune_checkpoint = "./tensorflow/workspace/pretrained_models/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0"
pconfig.train_config.fine_tune_checkpoint_type = "detection"
pconfig.train_input_reader.label_map_path= "./tensorflow/workspace/annotations/label_map.pbtxt"
pconfig.train_input_reader.tf_record_input_reader.input_path[:] = ["./tensorflow/workspace/annotations/train.record"]
pconfig.eval_input_reader[0].label_map_path = "./tensorflow/workspace/annotations/label_map.pbtxt"
pconfig.eval_input_reader[0].tf_record_input_reader.input_path[:] = ["./tensorflow/workspace/annotations/test.record"]

config = text_format.MessageToString(pconfig)

with tf.io.gfile.GFile("./tensorflow/workspace/models/model/pipeline.config", "wb") as f:
    f.write(config)

## Step 8: Train!

It's finally time to run the training program.

In [ ]:
# If we have already trained, we may want to skip this step.
# If you have not collected trained yet, set this to False.
skipTraining = True

if not skipTraining:
    !echo "It's training time!"
    !python ./models/research/object_detection/model_main_tf2.py                 \
     --model_dir=./tensorflow/workspace/models/model/                            \
     --pipeline_config_path=./tensorflow/workspace/models/model/pipeline.config  \
     --num_train_steps=2000
    !echo "Done training."